# Dynamic Programming On Tree
## What Is A Tree?
A tree is a DAG with one root, where the root has no parent. All other notes has and only has one parent.          
On a Tree, the path from root to any node is unique.

## DP on Tree
The dependency in Tree DP is relatively simple compare to other DP, since most of the relation is **Parent Depends On Children**

## Usual Methodology
1. Analyze what information does the parent need from the children
2. Pack all information needed as the return value of recursion
3. Use recursion to let parent get information from children

---
### Q1. Largest BST Subtree (LC.333)
*Given the root of a binary tree, find the largest subtree, which is also a Binary Search Tree (BST), where the largest means subtree has the largest number of nodes.*

*A Binary Search Tree (BST) is a tree in which all the nodes follow the below-mentioned properties:*

- *The left subtree values are less than the value of their parent (root) node's value.*
- *The right subtree values are greater than the value of their parent (root) node's value.*

*Note: A subtree must include all of its descendants.*

**Solution:**         
For any subtree with root node `X`, X can either be a BST or not. 
- If X is a BST, then the largest BST subtree is itself.
- If X is not a BST, then the largest BST subtree of X is the largest BST subtree of left or right
   
What information from children do we need to know if X is a BST?
- isBST(left)
- isBST(right)
- The max value from left child
- The min value from right child
- The size of the largest BST subtree on left
- The size of the largest BST subtree on right

If `isBST(left)` and `isBST(right)` and `max(left) < X.val < min(right)`, we know that X is the largest BST.      
And if X is not a BST, we compare the size of the largest BST on left and right.

In [18]:
class Solution:
    class Info:
        def __init__(self, isBST, maxVal, minVal, maxBSTSize):
            self.isBST = isBST
            self.maxVal = maxVal
            self.minVal = minVal
            self.maxBSTSize = maxBSTSize

    def largestBSTSubtree(self, root):
        def dfs(root):
            if not root:
                return self.Info(True, float('-inf'), float('inf'), 0)

            leftInfo = dfs(root.left)
            rightInfo = dfs(root.right)
            maxVal = max(root.val, leftInfo.maxVal, rightInfo.maxVal)
            minVal = min(root.val, leftInfo.minVal, rightInfo.minVal)
            isBST = leftInfo.isBST and rightInfo.isBST and leftInfo.maxVal < root.val < rightInfo.minVal

            if isBST:
                maxBSTSize = leftInfo.maxBSTSize + rightInfo.maxBSTSize + 1
            else:
                maxBSTSize = max(leftInfo.maxBSTSize, rightInfo.maxBSTSize)

            return self.Info(isBST, maxVal, minVal, maxBSTSize)

        return dfs(root).maxBSTSize

---
### Q2. Maximum Sum BST In Binary Tree (LC.1373)
*Given a binary tree root, return the maximum sum of all keys of any sub-tree which is also a Binary Search Tree (BST).*

**Solution:**   
Very similar to Q1, but note that we need another info: the sum of all the nodes in a subtree. Because for any subtree, `maxBSTSum` might not be the actual sum of the whole subtree since there are negative value in nodes.    
Also, in python you don't need a class. You can just return a tuple for all of the info.

In [11]:
class Solution:
    def maxSumBST(self, root):
        def dfs(node):
            if not node:
                # (isBST, maxVal, minVal, maxSum, totalSum)
                return (True, float('-inf'), float('inf'), 0, 0)

            leftIsBST, leftMax, leftMin, leftMaxSum, leftTotalSum = dfs(node.left)
            rightIsBST, rightMax, rightMin, rightMaxSum, rightTotalSum = dfs(node.right)

            isBST = leftIsBST and rightIsBST and leftMax < node.val < rightMin
            totalSum = leftTotalSum + rightTotalSum + node.val
            maxSum = max(leftMaxSum, rightMaxSum)

            if isBST:
                maxSum = max(maxSum, totalSum)

            maxVal = max(node.val, leftMax, rightMax)
            minVal = min(node.val, leftMin, rightMin)

            return (isBST, maxVal, minVal, maxSum, totalSum)

        return dfs(root)[3]

---
### Q3. Diameter Of Binary Tree (LC.543)
*Given the root of a binary tree, return the length of the diameter of the tree.*

*The diameter of a binary tree is the length of the longest path between any two nodes in a tree. This path may or may not pass through the root.*

*The length of a path between two nodes is represented by the number of edges between them.*

**Solution:**     
Again, two cases:
- If the diameter of a subtree doesn't go through its root, then it's the max diameter of its two children
- If the diameter of a subtree go through its root, then it is `height(left) + height(right) + 1`

In [16]:
class Solution:
    def diameterOfBinaryTree(self, root):
        def dfs(root):
            if not root:
                return (0, 0) #(Diameter, Height)
            leftD, leftH = dfs(root.left)
            rightD, rightH = dfs(root.right)
            h = max(leftH, rightH) + 1
            d = max(leftH + rightH, max(leftD, rightD))
            return (d, h)
        return dfs(root)[0]

---
### Q4. Distribute Coins In Binary Tree
*You are given the root of a binary tree with n nodes where each node in the tree has `node.val` coins. There are `n` coins in total throughout the whole tree.*

*In one move, we may choose two adjacent nodes and move one coin from one node to another. A move may be from parent to child, or from child to parent.*

*Return the minimum number of moves required to make every node have exactly one coin.*

**Solution:**          
Lets use an example to analyze the problem: suppose we have a subtree that has 20 nodes and 40 coins. This means that we must move 20 coins to the root, then to either the right child or the parent. Similarly, if the right child has 20 nodes and 10 coins, then we need to move 10 coins to it, needing 10 operations.     
Therefore, for any subtree, we need `|left.numNode - left.numCoins| + |right.numNodes - right.numCoins|` operations.          
For the same root node in our example, left has 20 extra coins and right need 10 coins, thus there will be a total of 10 coins that we need to give to the parent node.


In [27]:
class Solution:

    def distributeCoins(self, root):
        numOperations = 0
        
        def dfs(root):
            nonlocal numOperations
            if not root:
                return (0, 0) #(numNodes, numCoins)
            lNodes, lCoins = dfs(root.left)
            rNodes, rCoins = dfs(root.right)
            numNodes = lNodes + rNodes + 1
            numCoins = lCoins + rCoins + root.val
            numOperations += abs(lNodes - lCoins) + abs(rNodes - rCoins)
            return (numNodes, numCoins)
            
        dfs(root)
        return numOperations

---
### Q5. House Robber III (LC.337)
*The thief has found himself a new place for his thievery again. There is only one entrance to this area, called root.*

*Besides the root, each house has one and only one parent house. After a tour, the smart thief realized that all houses in this place form a binary tree. It will automatically contact the police if two directly-linked houses were broken into on the same night.*

*Given the root of the binary tree, return the maximum amount of money the thief can rob without alerting the police.*

**Solution:**
Again, for a subtree we can either use the root or not.           
- If we use root, then you cannot use both children        
- If we don't use root, then we can use the children (**OR NOT!!**)
      
Therefore, for each root we need two information from each child.
- The max money we can stole without robbing this node
- The max money we can stole if we rob this node

In [9]:
class Solution:
    def rob(self, root):
        
        def dfs(root):
            if not root:
                return (0, 0)
            robLeft, notRobLeft = dfs(root.left)
            robRight, notRobRight = dfs(root.right)
            robRoot = root.val + notRobLeft + notRobRight
            # Note that even we didn't rob root, we may not want to rob child because we might rob the grandchild
            notRobRoot = max(robLeft, notRobLeft) + max(robRight, notRobRight)
            return (robRoot, notRobRoot)
            
        return max(dfs(root))

---
### Q6. Binary Tree Cameras (LC.968)
*You are given the root of a binary tree. We install cameras on the tree nodes where each camera at a node can monitor its parent, itself, and its immediate children.*

*Return the minimum number of cameras needed to monitor all nodes of the tree.*

**Solution:**    
For a node `x`, there are essentially 3 status. The assumption here is that the node has a parent:
1. `x` is not covered
2. `x` is covered, but there isn't a camera on it
3. `x` is covered, and there is a camera on it

Depends on the status of children, we can decide whether we put camera on the current node or not:
- If any child is not covered, we need to put a camera on curNode, then return 3
- If both children is covered but current node is not, we don't put a camera and try to let the parent node take care of the current node, so return 1
- Otherwise, it means that both children is covered and at least one children has a camera on it, so the current node is covered. Thus we return 2.

Note that Null is case 2.      

In the end, because we assume every node has a parent, we need to handle the root node. If dfs(root) returns 1, it means that the root node is not covered. And since root has no parent, we need to put a camera on it.

In [18]:
class Solution:
    def minCameraCover(self, root):
        numCamera = 0
    
        def dfs(root):
            nonlocal numCamera
            if not root:
                return 2  # Null is case 2
            lStatus = dfs(root.left)
            rStatus = dfs(root.right)
            if lStatus == 1 or rStatus == 1:
                numCamera += 1
                return 3
            elif lStatus == 2 and rStatus == 2:
                return 1
            else:
                return 2
        
        if dfs(root) == 1:
            numCamera += 1
        return numCamera

---
### Q7. Path Sum III (LC.437)
*Given the root of a binary tree and an integer targetSum, return the number of paths where the sum of the values along the path equals targetSum.*

*The path does not need to start or end at the root or a leaf, but it must go downwards (i.e., traveling only from parent nodes to child nodes).*

**Solution:**       
Instead of letting the child return its info to the parent, this time we let parent pass their info to the child.     
When a node is reached, we need two information: the prefix sum of the path from root to this node, and presums from root to every parent node among this path.       
Therefore, if the current prefix sum is `x` and there exist a previous presum of `x - targetSum`, then we find a path of sum `targetSum`.
Thus we use a hashmap to store the presums, then perform backtracking algo.

In [3]:
class Solution:
    def pathSum(self, root):
        preSum = defaultdict(int)
        preSum[0] = 1 #Empty path makes sum 0
        cnt = 0

        def dfs(root, curSum):
            nonlocal preSum, cnt
            if not root: 
                return
            curSum += root.val
            cnt += preSum[curSum - targetSum]
            preSum[curSum] += 1

            dfs(root.left, curSum)
            dfs(root.right, curSum)

            # Backtrack
            preSum[curSum] -= 1

        dfs(root, 0)
        return cnt

---
### Q8. Minimum Cost To Report To The Capital
*There is a tree (i.e., a connected, undirected graph with no cycles) structure country network consisting of n cities numbered from 0 to n - 1 and exactly n - 1 roads. The capital city is city 0. You are given a 2D integer array roads where roads[i] = [ai, bi] denotes that there exists a bidirectional road connecting cities ai and bi.*

*There is a meeting for the representatives of each city. The meeting is in the capital city.*

*There is a car in each city. You are given an integer seats that indicates the number of seats in each car.*

*A representative can use the car in their city to travel or change the car and ride with another representative. The cost of traveling between two cities is one liter of fuel.*

*Return the minimum number of liters of fuel to reach the capital city.*

**Solution:**
Similar to Q1-6, pass here

In [11]:
class Solution:
    def minimumFuelCost(self, roads, seats):
        graph = defaultdict(list)
        for src, des in roads:
            graph[src].append(des)
            graph[des].append(src)
        
        def dfs(node, parent):
            num_ppl, fuel_cost = 1, 0  # 1 person (the representative at `node`)
            for neighbor in graph[node]:
                if neighbor == parent:
                    continue  # Avoid going back to the parent node
                neighbor_ppl, neighbor_cost = dfs(neighbor, node)
                num_ppl += neighbor_ppl
                fuel_cost += neighbor_cost + (neighbor_ppl + seats - 1) // seats  # Calculate trips required
            return num_ppl, fuel_cost

        # Start DFS from the capital city (node 0) with no parent (-1)
        return dfs(0, -1)[1]


---
### Q9. Longest Path With Different Adjacent Characters (LC.2246)
*You are given a tree (i.e. a connected, undirected graph that has no cycles) rooted at node 0 consisting of n nodes numbered from 0 to n - 1. The tree is represented by a 0-indexed array parent of size n, where parent[i] is the parent of node i. Since node 0 is the root, parent[0] == -1.*

*You are also given a string s of length n, where s[i] is the character assigned to node i.*

*Return the length of the longest path in the tree such that no pair of adjacent nodes on the path have the same character assigned to them.*

In [16]:
class Solution:
    def longestPath(self, parent):
        from collections import defaultdict

        # Build the tree as an adjacency list
        children = defaultdict(list)
        n = len(parent)
        for i in range(1, n):  # Skip root since parent[0] == -1
            children[parent[i]].append(i)

        self.max_length = 0  # Track the longest path globally

        def dfs(node):
            max1, max2 = 0, 0  # Top two longest paths from children
            
            for child in children[node]:
                child_len = dfs(child)
                # Only consider the child path if the characters are different
                if s[node] != s[child]:
                    if child_len > max1:
                        max1, max2 = child_len, max1
                    elif child_len > max2:
                        max2 = child_len
            
            # Update global maximum path (including current node)
            self.max_length = max(self.max_length, max1 + max2 + 1)
            
            # Return the longest single path including this node
            return max1 + 1

        dfs(0)  # Start DFS from the root node
        return self.max_length

# DFN (Depth-First Number)
DFN is essentially used to give each node an ID based on dfs traversal.    

DFN will give us this property:
- The id of a parent node must be smaller than id of a child

Thus given the `DFN` of the root of a subtree and the `size` of the subtree, we can locate all nodes in the subtree
- Suppose `DFN[root] == x` and `size[root] == y`, then we know that all nodes with DFN from `x` to `x + y - 1` are on this same subtree

Therefore, we can use DFN to tell **whether a given node is on a given subtree**

In [ ]:
class DFN:
    def __init__(self, id, dfn):
        self.id = 0
        self.dfn = []

    def dfs(node):
        dfn[node] = id
        id += 1
        for child in graph[node]:
            dfs(node)

---
### Q9. Height Of Binary Tree After Subtree Removal Queries (LC.2458) - DFN Template Problem
*You are given the root of a binary tree with n nodes. Each node is assigned a unique value from 1 to n. You are also given an array queries of size m.*

*You have to perform m independent queries on the tree where in the ith query you do the following:*

- *Remove the subtree rooted at the node with the value queries[i] from the tree. It is guaranteed that queries[i] will not be equal to the value of the root.* 
- *Return an array answer of size m where answer[i] is the height of the tree after performing the ith query.*

*Note:*
- *The queries are independent, so the tree returns to its initial state after each query.*
- *The height of a tree is the number of edges in the longest simple path from the root to some node in the tree.*

**Solution:**      
By running a DFS on the tree, we can construct 3 arrays:
- `DFN[val]`: the index is node.val, and the value is the corresponding DFN for the node
- `size[dfn]`: the index is the DFN of the node, the value is the size of subtree
- `depth[dfn]`: the index is the DFN of the node, the value is the depth of the node

The height of the whole tree is then `max(depth)`.

Now suppose we have a query: remove(6).
- We first use `DFN[6]` to find the dfn of the node, lets suppose it's 4
- Then we use `size[4]` to find the size of the subtree, suppose its 3
- Now knowing the size of the subtree, we know that node with dfn 4, 5, 6 are all removed from the whole tree
- Then we just need to find max(depth) without `depth[4]`, `depth[5]`, and `depth[6]`

To find the max(depth) without a continuous range, we can use two auxiliary array, maxL and maxR:
- `maxL[i]`: the maximum depth from dfn `0` to dfn `i`
- `maxR[i]`: the maximum depth from dfn `i` to dfn `n - 1`

Now, to find max(depth) without dfn 4, 5, and 6, we just need to find `max(maxL[4], maxR[6])`

**Time Complexity: O(n + m)**

In [29]:
class Solution:
    def treeQueries(self, root, queries):
        MAXN = 100002
        cnt = 0                 # dfn start with 1
        dfn = [0] * MAXN
        size = [0] * MAXN
        depth = [0] * MAXN
        maxL = [0] * MAXN       # max depth in [1, i]
        maxR = [0] * MAXN       # max depth in [i, cnt]

        def dfs(node, k):
            nonlocal cnt
            cnt += 1
            i = cnt
            dfn[node.val] = i
            depth[i] = k
            size[i] = 1  # Include the node itself
            if node.left:
                dfs(node.left, k + 1)
                size[i] += size[dfn[node.left.val]]
            if node.right:
                dfs(node.right, k + 1)
                size[i] += size[dfn[node.right.val]]

        dfs(root, 0)

        for i in range(1, cnt + 1):
            maxL[i] = max(maxL[i - 1], depth[i])
        for i in range(cnt, 0, -1):
            maxR[i] = max(maxR[i + 1], depth[i])

        ans = []
        for q in queries:
            leftMax = maxL[dfn[q] - 1]
            rightMax = maxR[dfn[q] + size[dfn[q]]]
            ans.append(max(leftMax, rightMax))
        
        return ans

---
### Q10. Minimum Score After Removals On A Tree (LC.2322)
*There is an undirected connected tree with n nodes labeled from 0 to n - 1 and n - 1 edges.*

*You are given a 0-indexed integer array nums of length n where nums[i] represents the value of the ith node. You are also given a 2D integer array edges of length n - 1 where edges[i] = [ai, bi] indicates that there is an edge between nodes ai and bi in the tree.*

*Remove two distinct edges of the tree to form three connected components. For a pair of removed edges, the following steps are defined:*
- *Get the XOR of all the values of the nodes for each of the three components respectively.*
- *The difference between the largest XOR value and the smallest XOR value is the score of the pair.*

*For example, say the three components have the node values: [4,5,7], [1,9], and [3,3,3]. The three XOR values are 4 ^ 5 ^ 7 = 6, 1 ^ 9 = 8, and 3 ^ 3 ^ 3 = 3. The largest XOR value is 8 and the smallest XOR value is 3. The score is then 8 - 3 = 5.*        

*Return the minimum score of any possible pair of edge removals on the given tree.*

**Solution:**
From the data size constraint (numEdges <= 1000) we know that a O(n^2) method can work. Therefore we can try any combination of two edges to delete.    
The key question is how can we quickly calculate the XOR of the 3 components.       
There are 3 cases, check video!